In [43]:
from sklearn.cluster import  MiniBatchKMeans, KMeans, DBSCAN
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import pandas as pd
import math
from pylab import *
from scipy.spatial.distance import cdist
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import ConvexHull
from geojson import Polygon
from geojson import MultiPolygon
import random
from scipy import spatial
import mpld3
import json
import glob

In [10]:
from logistics_functions import *
from logistics_model import logistics_model

In [11]:
import folium
import json
from IPython.display import HTML

In [12]:
path = '..\shapefiles\\building_block_people_WGS_3.csv'
n_regen = 30
n_collection = 1

In [13]:
building_virtual_buildings_df = get_virtual_buildings(path)
building_SF_points=building_virtual_buildings_df[['lat_lat','lon_lon']]

In [14]:
k_means_labels_regen, k_means_cluster_centers_regen = clustering(building_SF_points, n_regen)
k_means_labels_collection, k_means_cluster_centers_collection = clustering(k_means_cluster_centers_regen, n_collection)

In [15]:
len(k_means_cluster_centers_regen)

30

fig = plt.figure(figsize=(8, 8))
colors = plt.cm.Spectral(np.linspace(0, 1, n_regen))
ax = fig.add_subplot(1, 1, 1)

data_plot=np.asarray(building_SF_points[:])
for k, col in zip(range(n_regen), colors):
    my_members = k_means_labels_regen == k
    cluster_center = k_means_cluster_centers_regen[k]
    plt.plot(data_plot[my_members, 1], data_plot[my_members, 0], 'w',
            markerfacecolor=col, marker='o', markersize=6)
    plt.plot(cluster_center[1], cluster_center[0], 's',
            markerfacecolor='r', markeredgecolor='k', markersize=10)    
plt.title('KMeans Clustering',fontsize=18)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.show()

In [16]:
size_all= []
hull_all =[]
patches =[]
colors = plt.cm.nipy_spectral(np.linspace(0, 1, n_regen))
colors=colors.tolist()
for i in range (k_means_labels_regen.max()+1):
    my_members = k_means_labels_regen==i
    unique = find_unique(building_virtual_buildings_df[my_members],'gid')
    size_all.append(unique)
    
    lat_lon = size_all[i][['lat_lat', 'lon_lon']]
    lat_lon_array = np.array(lat_lon)
    hull = ConvexHull(lat_lon_array)
    lat_lon_array = np.array(size_all[i][['lat_lat', 'lon_lon']])
    polygon_coords=[]
    for simplex in hull.vertices:
        x= lat_lon_array[simplex, 1]
        y = lat_lon_array[simplex, 0]
        coords = (x,y)
        polygon_coords.append(coords)
    polygon_coords_array = [polygon_coords]
    polygon = Polygon(polygon_coords_array)
    patches.append(polygon)


In [17]:
ice_map = folium.Map(location=[37.6236, -122.3750])
from random import randint
from matplotlib.colors import rgb2hex

i=0
for item in patches:
    with open('rest_%s.json' %i, 'w') as outfile: 
        json.dump(item, outfile)
    i=i+1
for z in range(i):
    color = matplotlib.colors.rgb2hex(colors[z])
    ice_map.geo_json(geo_path='rest_%s.json' %z, fill_color=color, fill_opacity=0.5, line_opacity=0.9)
#ice_map.create_map(path='ice_map_test_2.html')
z=0
for item in k_means_cluster_centers_regen:
    color = matplotlib.colors.rgb2hex(colors[z])
    folium.RegularPolygonMarker(
    item,
    popup='Hawthorne Bridge',
    fill_color=color,
    number_of_sides=4,
    radius=7
    ).add_to(ice_map)
    z=z+1
for item in k_means_cluster_centers_collection:    
    folium.RegularPolygonMarker(
    item,
    popup='Hawthorne Bridge',
    fill_color='#00FFFF',
    number_of_sides=6,
    radius=10
    ).add_to(ice_map)

ice_map

C:\Users\Olga\Anaconda2\lib\site-packages\folium\folium.py:504: UserWarning: This method is deprecated. Please use Map.choropleth instead.
  warnings.warn('This method is deprecated. '


In [18]:
#map_osm.save("cluster_30map.html")

In [19]:
nx = int(math.sqrt(n_regen))
ny = int(n_regen/nx)
grid_coord = make_grid_points(nx, ny)

In [20]:
grid_map = folium.Map(location=[37.6236, -122.3750])
for item in grid_coord:    
    folium.RegularPolygonMarker(
    item,
    popup='Hawthorne Bridge',
    fill_color='red',
    number_of_sides=4,
    radius=7
    ).add_to(grid_map)
grid_map

In [21]:
cluster_centers_regen_df=pd.DataFrame(grid_coord, columns=['lat', 'lon'])
tree = spatial.KDTree(zip(cluster_centers_regen_df['lat'], cluster_centers_regen_df['lon']))
building_SF_points_array = zip(building_SF_points['lat_lat'],building_SF_points['lon_lon'])
k_means_labels_regen = tree.query(building_SF_points_array)[1]
labels_collection, cluster_centers_collection = clustering(grid_coord, n_collection)

fig = plt.figure(figsize=(8, 8))
colors = plt.cm.Spectral(np.linspace(0, 1, n_regen))
ax = fig.add_subplot(1, 1, 1)

data_plot=np.asarray(building_SF_points[:])
for k, col in zip(range(n_regen), colors):
    my_members = k_means_labels_regen == k
    cluster_center = grid_coord[k]
    plt.plot(data_plot[my_members, 1], data_plot[my_members, 0], 'w',
            markerfacecolor=col, marker='o', markersize=6)
    plt.plot(cluster_center[1], cluster_center[0], 's',
            markerfacecolor='r', markeredgecolor='k', markersize=10)    
plt.title('KMeans Clustering',fontsize=18)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.show()

In [290]:
size_all= []
hull_all =[]
patches =[]
colors = plt.cm.nipy_spectral(np.linspace(0, 1, n_regen+1))
colors=colors.tolist()
for i in range (k_means_labels_regen.max()+1):
    my_members = k_means_labels_regen==i
    unique = find_unique(building_virtual_buildings_df[my_members],'gid')
    size_all.append(unique)
    
    lat_lon = size_all[i][['lat_lat', 'lon_lon']]
    lat_lon_array = np.array(lat_lon)
    hull = ConvexHull(lat_lon_array)
    lat_lon_array = np.array(size_all[i][['lat_lat', 'lon_lon']])
    polygon_coords=[]
    for simplex in hull.vertices:
        x= lat_lon_array[simplex, 1]
        y = lat_lon_array[simplex, 0]
        coords = (x,y)
        polygon_coords.append(coords)
    polygon_coords_array = [polygon_coords]
    polygon = Polygon(polygon_coords_array)
    patches.append(polygon)

In [291]:
ice_map = folium.Map(location=[37.6236, -122.3750])

i=0
for item in patches:
    with open('rest_%s_.json' %i, 'w') as outfile: 
        json.dump(item, outfile)
    i=i+1
for z in range(i):
    color = matplotlib.colors.rgb2hex(colors[z])
    ice_map.geo_json(geo_path='rest_%s_.json' %z, fill_color=color, fill_opacity=0.5, line_opacity=0.9)
#ice_map.create_map(path='ice_map_test_2.html')
z=0
for item in grid_coord:
    color = matplotlib.colors.rgb2hex(colors[z])
    folium.RegularPolygonMarker(
    item,
    popup='Hawthorne Bridge',
    fill_color=color,
    number_of_sides=4,
    radius=7
    ).add_to(ice_map)
    z=z+1
for item in cluster_centers_collection:    
    folium.RegularPolygonMarker(
    item,
    popup='Hawthorne Bridge',
    fill_color='#00FFFF',
    number_of_sides=6,
    radius=10
    ).add_to(ice_map)

ice_map

In [24]:
cluster_centers_regen = make_random_points(n_regen)

In [25]:
grid_map = folium.Map(location=[37.6236, -122.3750])
for item in cluster_centers_regen:    
    folium.RegularPolygonMarker(
    item,
    popup='Hawthorne Bridge',
    fill_color='red',
    number_of_sides=4,
    radius=7
    ).add_to(grid_map)
grid_map

In [26]:
cluster_centers_regen_df=pd.DataFrame(cluster_centers_regen, columns=['lat', 'lon'])
tree = spatial.KDTree(zip(cluster_centers_regen_df['lat'], cluster_centers_regen_df['lon']))
building_SF_points_array = zip(building_SF_points['lat_lat'],building_SF_points['lon_lon'])
k_means_labels_regen = tree.query(building_SF_points_array)[1]
labels_collection, cluster_centers_collection = clustering(cluster_centers_regen, n_collection)

fig = plt.figure(figsize=(8, 8))
colors = plt.cm.Spectral(np.linspace(0, 1, n_regen))
ax = fig.add_subplot(1, 1, 1)

data_plot=np.asarray(building_SF_points[:])
for k, col in zip(range(n_regen), colors):
    my_members = k_means_labels_regen == k
    cluster_center = cluster_centers_regen[k]
    plt.plot(data_plot[my_members, 1], data_plot[my_members, 0], 'w',
            markerfacecolor=col, marker='o', markersize=6)
    plt.plot(cluster_center[1], cluster_center[0], 's',
            markerfacecolor='r', markeredgecolor='k', markersize=10)    
plt.title('KMeans Clustering',fontsize=18)
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.show()

In [134]:
size_all= []
hull_all =[]
patches = []
colors = plt.cm.nipy_spectral(np.linspace(0, 1, n_regen+1))
colors=colors.tolist()
for i in range (k_means_labels_regen.max()+1):
    my_members = k_means_labels_regen==i
    unique = find_unique(building_virtual_buildings_df[my_members],'gid')
    size_all.append(unique)
    
    lat_lon = size_all[i][['lat_lat', 'lon_lon']]
    lat_lon_array = np.array(lat_lon)
    hull = ConvexHull(lat_lon_array)
    lat_lon_array = np.array(size_all[i][['lat_lat', 'lon_lon']])
    polygon_coords=[]
    for simplex in hull.vertices:
        x= lat_lon_array[simplex, 1]
        y = lat_lon_array[simplex, 0]
        coords = (x,y)
        polygon_coords.append(coords)
    polygon_coords_array = [polygon_coords]
    polygon = Polygon(polygon_coords_array)
    patches.append(polygon)

In [29]:
ice_map = folium.Map(location=[37.6236, -122.3750])

i=0
for item in patches:
    with open('rest_%s_.json' %i, 'w') as outfile: 
        json.dump(item, outfile)
    i=i+1
for z in range(i):
    color = matplotlib.colors.rgb2hex(colors[z])
    ice_map.geo_json(geo_path='rest_%s_.json' %z, fill_color=color, fill_opacity=0.5, line_opacity=0.9)
#ice_map.create_map(path='ice_map_test_2.html')
z=0
for item in cluster_centers_regen:
    color = matplotlib.colors.rgb2hex(colors[z])
    folium.RegularPolygonMarker(
    item,
    popup='Hawthorne Bridge',
    fill_color=color,
    number_of_sides=4,
    radius=7
    ).add_to(ice_map)
    z=z+1
for item in cluster_centers_collection:    
    folium.RegularPolygonMarker(
    item,
    popup='Hawthorne Bridge',
    fill_color='#00FFFF',
    number_of_sides=6,
    radius=10
    ).add_to(ice_map)

ice_map

In [175]:
def df_to_geojson(df,properties, feature_type):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature', 'properties':{},
                   'geometry':{'type':feature_type,
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row['coordinates']]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [170]:
patches_df = pd.DataFrame()
patches_df_index = pd.DataFrame()
i = 0
for item in patches:
    coords = item
    coords_f = pd.DataFrame(coords)
    patches_df = patches_df.append(coords_f)

for index, row in patches_df.iterrows():
    row['cluster'] = int(i)
    i = i+1
    patches_df_index = patches_df_index.append(row)

In [178]:
geo = df_to_geojson(patches_df_index, ['cluster'], 'Polygon')

In [173]:
with open('patches_geo.json', 'w') as outfile: 
    json.dump(geo, outfile)

In [267]:
regen_centers_df = pd.DataFrame()
regen_centers_index = pd.DataFrame()
i = 0
for item in cluster_centers_regen:
    lat = item[0]
    lon = item[1]
    coor= [(lat,lon)]
    regen_centers_df = regen_centers_df.append(coor)

for index, row in regen_centers_df.iterrows():
    row['cluster'] = int(i)
    i = i+1
    regen_centers_index = regen_centers_index.append(row)
regen_centers_index.columns = ['lat', 'lon', 'cluster']

In [257]:
geo_points = df_to_geojson(regen_centers_index, ['cluster'], 'Point')

In [274]:
with open('points_geo_.json', 'w') as outfile: 
    json.dump(points, outfile)

In [271]:
def df_to_geojson_points(df, properties):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row['lon'],row['lat']]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

In [272]:
points = df_to_geojson_points(regen_centers_index, ['cluster'])